In [1]:
import numpy as np
import pandas as pd
from pandas import pivot_table
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import json

%matplotlib inline

In [3]:
post = pd.read_csv('postcodes.csv')

print(post.shape)
post.head()

C:\Users\liamk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,17,18,21,22,24,25,29,32,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2615669, 44)


,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force
0,AB1 0AA,No,57.101474,-2.242851,385386.0,801193.0,NJ853011,NaN,Aberdeen City,Lower Deeside,...,S14000002,6808.0,1.0,0.0,2019-05-29,Portlethen,8.31408,AB,AB1,Scotland
1,AB1 0AB,No,57.102554,-2.246308,385177.0,801314.0,NJ851013,NaN,Aberdeen City,Lower Deeside,...,S14000002,6808.0,1.0,0.0,2019-05-29,Portlethen,8.55457,AB,AB1,Scotland
2,AB1 0AD,No,57.100556,-2.248342,385053.0,801092.0,NJ850010,NaN,Aberdeen City,Lower Deeside,...,S14000002,6808.0,1.0,0.0,2019-05-29,Portlethen,8.54352,AB,AB1,Scotland
3,AB1 0AE,No,57.084444,-2.255708,384600.0,799300.0,NO845992,NaN,Aberdeenshire,North Kincardine,...,S14000058,5503.0,8.0,0.0,2019-05-29,Portlethen,8.20809,AB,AB1,Scotland
4,AB1 0AF,No,57.096656,-2.258102,384460.0,800660.0,NJ844006,NaN,Aberdeen City,Lower Deeside,...,S14000002,6668.0,8.0,1.0,2019-05-29,Portlethen,8.85583,AB,AB1,Scotland


In [7]:
pedit = post.copy()

pedit.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Postcode area',
       'Postcode district', 'Police force'],
      dtype='object')

In [19]:
# includes London; not geojson

print(len(pedit['County'].value_counts()))
pedit['County'].value_counts()[:5]

48


Greater London        313626
Greater Manchester    112130
West Yorkshire         87500
Hampshire              85506
West Midlands          84750
Name: County, dtype: int64

In [21]:
print(len(pedit['District'].value_counts()))
pedit['District'].value_counts()[:5]

382


Birmingham     34039
Westminster    31092
Leeds          29683
Manchester     26386
Cornwall       26210
Name: District, dtype: int64

In [17]:
# Districts include the 32 London boroughs
len(pedit[pedit['County'] == 'Greater London']['District'].value_counts())

32

In [28]:
pedit['District Code'].sample(5)

333574     N09000005
2585094    E07000168
1988778    W06000010
725037     E09000030
1272325          NaN
Name: District Code, dtype: object

In [32]:
len(pedit['Postcode district'].unique())

3091

In [33]:
pedit['Region'].value_counts()

South East                  402828
London                      320426
North West                  297128
South West                  243760
East of England             242526
Yorkshire and The Humber    214320
West Midlands               197177
East Midlands               167796
North East                   98962
Name: Region, dtype: int64

In [35]:
with open("Regions_December_2015_Ultra_Generalised_Clipped_Boundaries_in_England.geojson") as geofile:
    geojson_layer = json.load(geofile)

In [38]:
nuts1 = pd.read_csv('NUTS_Level_1_January_2018_Full_Clipped_Boundaries_in_the_United_Kingdom.csv')

nuts1.head()

,objectid,nuts118cd,nuts118nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC,North East (England),417313,600358,-1.72890,55.297031,8.592429e+09,1.008435e+06
1,2,UKD,North West (England),350015,506280,-2.77237,54.449451,1.416409e+10,2.075324e+06
2,3,UKE,Yorkshire and The Humber,446903,448736,-1.28712,53.932640,1.540912e+10,1.385386e+06
3,4,UKF,East Midlands (England),477660,322635,-0.84967,52.795719,1.564327e+10,1.332317e+06
4,5,UKG,West Midlands (England),386294,295477,-2.20358,52.556969,1.300374e+10,9.539014e+05
